# Tutorial 0: Getting started with TenSeal
[Source](https://github.com/OpenMined/TenSEAL/blob/main/tutorials/Tutorial%200%20-%20Getting%20Started.ipynb)

## Tenseal

In [15]:
import tenseal as ts
print(f'tenseal version: {ts.__version__}')

tenseal version: 0.3.5


TenSEAL supports addition, substraction and multiplication of encrypted vectors 
of either integers (using BFV) or real numbers (using CKKS).

In [23]:
context = ts.context(ts.SCHEME_TYPE.BFV, 
                    poly_modulus_degree=4096, 
                    plain_modulus=1032193)
context

Context holds different encryption keys and parameters so that you only need to use a single object 
to make your encrypted computation instead of managing all the keys and the HE details.

Check if the context is public or private

In [24]:
print("Is the context private?", ("Yes" if context.is_private() else "No"))
print("Is the context public?", ("Yes" if context.is_public() else "No"))
sk = context.secret_key()
print(sk)

# the context will drop the secret-key at this point
context.make_context_public()
print("Secret-key dropped")
print("Is the context private?", ("Yes" if context.is_private() else "No"))
print("Is the context public?", ("Yes" if context.is_public() else "No"))
try:
    sk = context.secret_key()
except Exception as e:
    print(f'Error: {e}')

Is the context private? Yes
Is the context public? No
Secret-key dropped
Is the context private? No
Is the context public? Yes
Error: the current context is public, it doesn't hold a Secret key


## Encryption and Evaluation

We have to construct the context again, as it needs to be private so we can decrypt 
the data after evaluation

In [29]:
context = ts.context(ts.SCHEME_TYPE.BFV, 
                    poly_modulus_degree=4096, 
                    plain_modulus=1032193)
context

In [34]:
plain_vector = [60, 66, 73, 81, 90]
encrypted_vector = ts.bfv_vector(context, plain_vector)
print("We just encrypted our plaintext vector of size:", encrypted_vector.size())
encrypted_vector.data

We just encrypted our plaintext vector of size: 5


Do addition, substraction and multiplication in an element-wise fashion 
with other encrypted or plain vectors.

In [40]:
add_result = encrypted_vector + [1, 2, 3, 4, 5]
print(add_result)
print(add_result.decrypt())

[61, 68, 76, 85, 95]


In [36]:
sub_result = encrypted_vector - [1, 2, 3, 4, 5]
print(sub_result.decrypt())

[59, 64, 70, 77, 85]


In [37]:
mul_result = encrypted_vector * [1, 2, 3, 4, 5]
print(mul_result.decrypt())

[60, 132, 219, 324, 450]


In [38]:
encrypted_add = add_result + sub_result
print(encrypted_add.decrypt())

[120, 132, 146, 162, 180]


In [39]:
encrypted_sub = encrypted_add - encrypted_vector
print(encrypted_sub.decrypt())

[60, 66, 73, 81, 90]


In [41]:
encrypted_mul = encrypted_add * encrypted_sub
print(encrypted_mul.decrypt())

[7200, 8712, 10658, 13122, 16200]


You should never encrypt your plaintext values to evaluate them with ciphertexts if 
they don't need to be kept private

In [43]:
from time import time

t_start = time()
_ = encrypted_add * encrypted_mul
t_end = time()
print("ciphertext and ciphertext multiply time: {} ms".format((t_end - t_start) * 1000))

t_start = time()
_ = encrypted_add * [1, 2, 3, 4, 5]
t_end = time()
print("ciphertext and plaintext multiply time: {} ms".format((t_end - t_start) * 1000))

ciphertext and ciphertext multiply time: 6.909370422363281 ms
ciphertext and plaintext multiply time: 0.8785724639892578 ms


## More on Tenseal context

Other attributes of context: setting automatic relinearization, 
rescaling (for CKKS only) and modulus switching. These features are enabled by 
defaut as you can see below:

In [44]:
print("Automatic relinearization is:", ("on" if context.auto_relin else "off"))
print("Automatic rescaling is:", ("on" if context.auto_rescale else "off"))
print("Automatic modulus switching is:", ("on" if context.auto_mod_switch else "off"))

Automatic relinearization is: on
Automatic rescaling is: off
Automatic modulus switching is: off


TenSEALContext can also hold a global_scale (only used when using CKKS), which is used as a default scale value when the user doesn't provide one. As most often users will define a single value to be used as scale during the entire HE computation, defining it globally can be more straight forward compared to passing it to every function call.

In [45]:
# this should throw an error as the global_scale isn't defined yet
try:
    print("global_scale:", context.global_scale)
except ValueError:
    print("The global_scale isn't defined yet")
    
# you can define it to 2 ** 20 for instance
context.global_scale = 2 ** 20
print("global_scale:", context.global_scale)

The global_scale isn't defined yet
global_scale: 1048576.0
